In [40]:
from tqdm.auto import tqdm
import numpy as np
import os
import pandas as pd
import sys

sys.path.append("..")

from src.lib.config import register_configs
from src.utils.EvalWrapper import EvalWrapper


# from eval_single_helper_funcs import instantiate_trainer, run_and_save, get_confidence_and_acc_single, get_temperature_file, get_distribution_file

In [61]:
# set global values:

experiments_singlelabel = {
#    "rema": "/gpfs/work/machnitz/plankton_logs/supervised/singlelabel/multirun/2022-04-19/08-14-50",
   "linear_singlelabel": "/gpfs/work/machnitz/plankton_logs/linear_eval/singlelabel/multirun/2022-04-29/08-31-47",
#    "finetune_singlelabel": "/gpfs/work/machnitz/plankton_logs/finetune/singlelabel/multirun/2022-04-19/08-15-24",
    "finetune_sgd_singlelabel": "/gpfs/work/machnitz/plankton_logs/finetune_sgd/singlelabel/multirun/2022-04-29/08-19-35"
}

vals = [1, 2.5, 5, 7.5]
data_splits_per_experiment = ",".join([f"{np.round(val * np.power(10, exponent), 15):.8f}" for exponent in np.arange(-7,0.1,1) for val in vals])
print("Data splits: ", data_splits_per_experiment)

Data splits:  0.00000010,0.00000025,0.00000050,0.00000075,0.00000100,0.00000250,0.00000500,0.00000750,0.00001000,0.00002500,0.00005000,0.00007500,0.00010000,0.00025000,0.00050000,0.00075000,0.00100000,0.00250000,0.00500000,0.00750000,0.01000000,0.02500000,0.05000000,0.07500000,0.10000000,0.25000000,0.50000000,0.75000000,1.00000000,2.50000000,5.00000000,7.50000000


In [42]:
def custom_accuracy(y_pred, y_true):
    
    _, preds =  torch.max(y_pred, dim=1)
    
    score = y_true == preds
    return score.int().mean()

# Single Label

In [43]:
best_checkpoints = []
return_metrics = dict()
for key, setup_path in experiments_singlelabel.items():
    experiment_folders = [os.path.join(setup_path, str(i)) for i in np.arange(0, 19)]
    return_metrics[key] = dict()
    # fig, axes = plt.subplots(nrows=4, ncols=5, figsize=(20, 15), sharex=True, sharey=True)
    for experiment_number, experiment_path in enumerate(tqdm(experiment_folders)):
        # print(experiment_number)
        with open(os.path.join(experiment_path, "main.log"), "r") as f:
            complete_log = f.readlines()
            found_best_checkpoint = False
            for line in complete_log:
                # print(line)
                if found_best_checkpoint:
                    best_checkpoint_part = os.path.normpath(line.strip().split("multirun/")[-1]).strip()
                    base_path = os.path.normpath(experiment_path.split("multirun")[0]).strip()
                    best_checkpoint_result = os.path.join(base_path, "multirun", best_checkpoint_part).strip()
                    # print(f"set best checkpoint to {best_checkpoint}")
                    break

                if "[main.main][INFO] - Best checkpoint path:" in line:
                    found_best_checkpoint = True
                    # print(f"found best checkpoint: {line}")
        best_checkpoints.append(best_checkpoint_result)


  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

In [5]:
os.path.join(base_path, best_checkpoint_part[1:])

'/gpfs/work/machnitz/plankton_logs/finetune_sgd/singlelabel/022-04-29/08-19-35/18/logs/checkpoints/epoch=01-v1.ckpt'

In [6]:
best_checkpoints

['/gpfs/work/machnitz/plankton_logs/linear_eval/singlelabel/multirun/2022-04-29/08-31-47/0/logs/checkpoints/epoch=25.ckpt',
 '/gpfs/work/machnitz/plankton_logs/linear_eval/singlelabel/multirun/2022-04-29/08-31-47/1/logs/checkpoints/epoch=21.ckpt',
 '/gpfs/work/machnitz/plankton_logs/linear_eval/singlelabel/multirun/2022-04-29/08-31-47/2/logs/checkpoints/epoch=25.ckpt',
 '/gpfs/work/machnitz/plankton_logs/linear_eval/singlelabel/multirun/2022-04-29/08-31-47/3/logs/checkpoints/epoch=10.ckpt',
 '/gpfs/work/machnitz/plankton_logs/linear_eval/singlelabel/multirun/2022-04-29/08-31-47/4/logs/checkpoints/epoch=27.ckpt',
 '/gpfs/work/machnitz/plankton_logs/linear_eval/singlelabel/multirun/2022-04-29/08-31-47/5/logs/checkpoints/epoch=22.ckpt',
 '/gpfs/work/machnitz/plankton_logs/linear_eval/singlelabel/multirun/2022-04-29/08-31-47/6/logs/checkpoints/epoch=12.ckpt',
 '/gpfs/work/machnitz/plankton_logs/linear_eval/singlelabel/multirun/2022-04-29/08-31-47/7/logs/checkpoints/epoch=21.ckpt',
 '/gpfs/

In [8]:

datamodule.setup(stage="fit")
val_dataloader = datamodule.val_dataloader()

for batch in val_dataloader:
    example_input, _ = batch
    break

val_logits, val_labels, val_return_metrics = run_and_save(best_checkpoint, val_dataloader, return_metrics,
                                                          key + "valid", experiment_number)

NameError: name 'datamodule' is not defined

In [ ]:

datamodule.setup(stage="test")
test_dataloader = datamodule.test_dataloader()

for batch in test_dataloader:
    example_input, _ = batch
    break

test_logits, test_labels, test_return_metrics = run_and_save(best_checkpoint, test_dataloader, return_metrics,
                                                             key + "test_new", experiment_number)

In [ ]:
reform = {(outerKey, innerKey): values for outerKey, innerDict in return_metrics.items() for innerKey, values in
          innerDict.items()}
single_label_df = pd.DataFrame.from_dict(reform).T
single_label_df.index.names = ("Model", "Experiment")
single_label_df

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
sns.lineplot(x="Data Fraction", y="Accuracy/Testing", hue="Model", style="Model", data=single_label_df, ax=ax,
             palette=hereon_color_array, markers=True, markersize=10, )

# for item in data_splits_per_experiment:
#     ax.text(item,0.3,f'{item * 100:.0f}%',color="grey", horizontalalignment="center", rotation=-45)

ax.set_xscale("log")
plt.savefig("Accuracies_Testing_Singlelabel.png", dpi=300)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
sns.lineplot(x="Data Fraction", y="ECE", hue="Model", style="Model", data=single_label_df, ax=ax,
             palette=hereon_color_array, markers=True, markersize=10, )

for item in data_splits_per_experiment:
    ax.text(item, 0.2, f'{item * 100:.0f}%', color="grey", horizontalalignment="center", rotation=-45)

ax.set_xscale("log")
ax.legend(loc="best")
plt.savefig("ECE_Testing_Singlelabel.png", dpi=300)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
sns.lineplot(x="Data Fraction", y="loss/Testing", hue="Model", style="Model", data=single_label_df, ax=ax,
             palette=hereon_color_array, markers=True, markersize=10, )

for item in data_splits_per_experiment:
    ax.text(item, 2, f'{item * 100:.0f}%', color="grey", horizontalalignment="center", rotation=-45)

ax.set_xscale("log")
ax.legend(loc="best")
plt.savefig("NLL_Testing_Singlelabel.png", dpi=300)